In [27]:
import re
import csv
import pandas as pd
import numpy as np
import math

In [28]:
def read_input_file(file_name,k):

    feature_vectors = []
    file = open(file_name,'r')

    for line in file:
        vectors = []
        data = re.split(r'[()\s]\s*', line)
        while '' in data:
            data.remove('')

        for item in data:
            vectors.append(int(item))
        feature_vectors.append(vectors)
    if k==1:
        with open("input.csv","w+") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            csvWriter.writerows(feature_vectors)
    else:
        with open("input_2.csv","w+") as my_csv:
            csvWriter = csv.writer(my_csv,delimiter=',')
            csvWriter.writerows(feature_vectors)        
read_input_file('ClassifiedSetData.txt',1)
read_input_file('TestSetData.txt',2)

In [29]:
class MLP:
    def __init__(self):
        self.number_of_nodes=10
        self.max_feature_value=96
        self.num_output_nodes=8
        self.True_Positive=[0]*8
        self.True_Negative=[0]*8
        self.False_Positive=[0]*8
        self.False_Negative=[0]*8
        
    def split_data(self):
        self.df=pd.read_csv('input.csv',header=None)
        self.df1=pd.read_csv('input_2.csv',header=None)
        self.df=self.df.rename({0:'ID',1:'Feature_1',2:'Feature_2',3:'Feature_3',4:'Feature_4',5:'Feature_5',6:'Feature_6',7:'Feature_7',8:'Feature_8',9:'Feature_9',10:'Feature_10',11:'Output'}, axis=1)
        self.df1=self.df1.rename({0:'ID',1:'Feature_1',2:'Feature_2',3:'Feature_3',4:'Feature_4',5:'Feature_5',6:'Feature_6',7:'Feature_7',8:'Feature_8',9:'Feature_9',10:'Feature_10',11:'Output'}, axis=1)
        self.train_data, self.validate_data, self.test_data = np.split(self.df.sample(frac=1,random_state=42), [int(.6*len(self.df)), int(.8*len(self.df))])
    def multiplication_between_layers(self,weights, inputs):
        total=0
        for i in range(len(weights)):
            total += (inputs[i] * weights[i])
        return total   
    def merge_data(self):
        frames = [self.train_data,self.validate_data]
        self.train_data=pd.concat(frames) 
    def logistic(self, x):
        return 1 / ( 1 + math.exp(-x))
    
    def normalize_array(self, features,normalize_value):
        normalized_array=[]
        for i in features:
            normalized_array.append(i/normalize_value)
        return normalized_array
    
    def initialize_weight(self,x_dim,y_dim):
        return np.random.randn(x_dim,y_dim)*np.sqrt(1/y_dim)
    
    def set_weights(self,Wh1,Wh2):
        self.Wh1=Wh1
        self.Wh2=Wh2
        
    def forward_propagation(self,training_data):
            

        self.hidden_layer = []
        for i in range(self.number_of_nodes):
            self.hidden_layer.append(self.logistic(self.multiplication_between_layers(self.Wh1[i], training_data)))

        self.output_layer = []
    
        for i in range(self.num_output_nodes):
            self.output_layer.append(self.logistic(self.multiplication_between_layers(self.Wh2[i], self.hidden_layer)))
        return self.hidden_layer, self.output_layer
 
    def back_propagation(self,learning_rate, min_accuracy):
        epochs = 0
        while self.accuracy(self.train_data) < min_accuracy:
            epochs += 1
            for value in self.train_data.values:
                training_vector = self.normalize_array(list(value)[1:11], self.max_feature_value)
                self.forward_propagation(training_vector)
   
                error_1= [] 
                for j in range(self.num_output_nodes):
                    p = self.output_layer[j]
                    learning_rate = 0.3
                    if j == list(value)[-1]:
                        learning_rate = 0.7
                    error_1.append(p * (1 - p) * (learning_rate - p))
  
                error_2 = [] 
                for j in range(self.number_of_nodes):
                    hidden_layer_input = self.hidden_layer[j]
                    output_with_error = self.multiplication_between_layers(self.Wh2[:,j], error_1)
                    error_2.append(hidden_layer_input* (1 - hidden_layer_input) * output_with_error)

                for j in range(self.number_of_nodes):
                    for i in range(self.num_output_nodes):
                        self.Wh2[i][j] = self.Wh2[i][j] + learning_rate * error_1[i] * self.hidden_layer[j]
                    for k in range(self.number_of_nodes):
                        self.Wh1[j][k] = self.Wh1[j][k]   + learning_rate * error_2[j] * training_vector[k]
            print('finish',self.s) # Vatsal this for just testing 
                 
        return epochs
    def accuracy(self, input_data):
        correct = 0
        for data in input_data.values:
            hidden_layer, output_layer = self.forward_propagation(self.normalize_array(list(data)[1:11], self.max_feature_value))
            if (np.argmax(output_layer) == list(data)[-1]):
                correct += 1
        self.s=correct / len(input_data)
        return correct / len(input_data)
    def confusion_matrix(self,input_data):
        number_of_correct=0
        accuracy=0
        for value in input_data.values:
            training_vector = self.normalize_array(list(value)[1:11], self.max_feature_value)
            hidden_layer,output_layer=self.forward_propagation(training_vector)
            model_output_pred=np.argmax(output_layer)
            actual_output=list(value)[-1]
            if(model_output_pred == actual_output):
                number_of_correct+=1
                self.True_Positive[actual_output]+=1
                j=0
                while j<8:
                    if j!=actual_output:
                        self.True_Negative[j]+=1
                    else:
                        self.False_Positive[model_output_pred]+=1
                        self.False_Positive[actual_output]+=1
                        for i in range(8):
                            if i!=model_output_pred or i!=actual_output:
                                self.False_Negative[i]+=1
                    j+=1
        accuracy = number_of_correct / len(input_data)
        return accuracy

    def confusion_matrix_print(self):
        print("True Positive: "+str(self.True_Positive))
        print("False Positive: "+str(self.False_Positive))
        print("True Negative: "+str(self.True_Negative))      
        print("False Negative: "+str(self.False_Negative))




d=MLP()
d.split_data()
Weights_input_layer=d.initialize_weight(10,10)
Weights_hidden_layer=d.initialize_weight(8,10)
d.merge_data()
d.set_weights(Weights_input_layer,Weights_hidden_layer)
print("Number of epochs:",d.back_propagation(0.3, 0.87))

finish 0.175
finish 0.2375
finish 0.2625
finish 0.2625
finish 0.25
finish 0.3
finish 0.3125
finish 0.325
finish 0.3125
finish 0.3125
finish 0.3125
finish 0.3125
finish 0.325
finish 0.35
finish 0.35
finish 0.35
finish 0.35
finish 0.35
finish 0.35
finish 0.3625
finish 0.3875
finish 0.4
finish 0.425
finish 0.425
finish 0.4375
finish 0.4375
finish 0.4375
finish 0.4375
finish 0.4375
finish 0.4625
finish 0.4625
finish 0.4625
finish 0.4625
finish 0.4625
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.475
finish 0.4875
finish 0.4875
finish 0.5
finish 0.5
finish 0.5
finish 0.5125
finish 0.5125
finish 0.5125
finish 0.525
finish 0.525
finish 0.525
finish 0.525
finish 0.55
finish 0.55
finish 0.55
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0.5625
finish 0

finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
finish 0.75
fini

finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825
finish 0.825

finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.8375
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
finish 0.85
fi

In [24]:
print("Holdout Accuracy: " + str(d.confusion_matrix(d.test_data)*100))
#print("Training Accuracy: " + str(accuracy(Training_Set)))

Holdout Accuracy: 40.0


In [25]:
d.confusion_matrix_print()

True Positive: [0, 1, 2, 1, 1, 1, 2, 0]
False Positive: [0, 2, 4, 2, 2, 2, 4, 0]
True Negative: [8, 7, 6, 7, 7, 7, 6, 8]
False Negative: [8, 7, 6, 7, 7, 7, 6, 8]


In [26]:
print("Holdout Accuracy: " + str(d.confusion_matrix(d.df1)*100))

Holdout Accuracy: 31.0
